### 요구사항

- 데이터 특성 파악하기
- 와인 품질에 영향을 미치는 요소 파악하기
- 데이터 전처리하기 
- decision tree 계열 알고리즘을 사용하여 분류 모델 학습시키고 교차 검증하기
- 테스트 셋에 대하여 인퍼런스 한 뒤, 플랫폼에 적어도 한번 결과 제출하기 

In [1]:
#데이터 특성 파악하기
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, KFold

train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')



In [2]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

def label_encode_type(df):
    df["type"] = label_encoder.fit_transform(df["type"]) 
    return df


In [3]:
def preprocess(df):
    df = label_encode_type(df)
    return df

In [4]:
preprocess(train_df)


,index,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,1
1,1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,0
2,2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,1
3,3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,1
4,4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5492,5492,5,7.7,0.150,0.29,1.3,0.029,10.0,64.0,0.99320,3.35,0.39,10.1,1
5493,5493,6,6.3,0.180,0.36,1.2,0.034,26.0,111.0,0.99074,3.16,0.51,11.0,1
5494,5494,7,7.8,0.150,0.34,1.1,0.035,31.0,93.0,0.99096,3.07,0.72,11.3,1
5495,5495,5,6.6,0.410,0.31,1.6,0.042,18.0,101.0,0.99195,3.13,0.41,10.5,1


In [5]:
preprocess(test_df)

,index,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,9.0,0.31,0.48,6.60,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,1
1,1,13.3,0.43,0.58,1.90,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,0
2,2,6.5,0.28,0.27,5.20,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,1
3,3,7.2,0.15,0.39,1.80,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,1
4,4,6.8,0.26,0.26,2.00,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,7.1,0.59,0.02,2.30,0.082,24.0,94.0,0.99744,3.55,0.53,9.7,0
996,996,8.7,0.15,0.30,1.60,0.046,29.0,130.0,0.99420,3.22,0.38,9.8,1
997,997,8.8,0.66,0.26,1.70,0.074,4.0,23.0,0.99710,3.15,0.74,9.2,0
998,998,7.0,0.42,0.19,2.30,0.071,18.0,36.0,0.99476,3.39,0.56,10.9,0


In [6]:
preprocessed_train_df = preprocess(train_df)
preprocessed_test_df = preprocess(test_df)

preprocessed_train_df.to_csv('./data/wine_train_preprocessed.csv', index = False)
preprocessed_test_df.to_csv('./data/wine_test_preprocessed.csv', index = False)


train_df = pd.read_csv('./data/wine_train_preprocessed.csv')
test_df = pd.read_csv('./data/wine_test_preprocessed.csv')

y_train = train_df["quality"]
x_train = train_df.drop(["index", "quality"], axis=1)
x_test = test_df.drop(["index"], axis=1)


In [7]:
from sklearn.model_selection import StratifiedKFold,cross_val_score

straified_kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=1234)

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


model = RandomForestClassifier(
    criterion = 'gini',
    n_estimators=210,
    max_depth=90,
    max_features = 3,
    min_samples_leaf=1,
    min_samples_split=4,
    random_state=1234
)


model.fit(x_train, y_train)

y_test = model.predict(x_test)

print("Accuracy: ", accuracy_score(y_test, y_pred))

NameError: name 'y_test' is not defined

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# model definition
model = RandomForestClassifier(
    criterion = 'gini',
    n_estimators=210,
    max_depth=90,
    max_features = 3,
    min_samples_leaf=1,
    min_samples_split=4,
    random_state=1234
)

# model training
model.fit(x_train, y_train)

# inference
y_pred = model.predict(x_test)

# evaluate model
print("Accuracy: ", accuracy_score(y_test, y_pred))


In [ ]:
import pandas as pd

# Create DataFrame from prediction results
df = pd.DataFrame({
    "index": x_test.index,  # assuming x_test is a DataFrame and has an index
    "quality": y_pred
})

# Save DataFrame to csv
df.to_csv('./data/predictions_2.csv', index=False)


In [34]:
import numpy as np
from numpy.random import randint
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.ensemble import RandomForestClassifier

rs_space={
    
    
    
  'max_depth': list(np.arange(10, 100, step=10)) + [None],
    'n_estimators': np.arange(100, 1000, step=100),
    'max_features': uniform(loc=0.1, scale=0.9),  # 변경: 범위 축소
    'criterion': ['gini', 'entropy'],
    'min_samples_leaf': np.arange(1, 10),
    'min_samples_split': np.arange(2, 20, step=2)
}

rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(rf, rs_space, n_iter=500, scoring='accuracy', n_jobs=-1, cv=3)
model_random = rf_random.fit(x_train, y_train)





KeyboardInterrupt: 

In [ ]:

# 최적의 하이퍼파라미터 확인
print(model_random.best_params_)

# 최고 정확도 확인
print(model_random.best_score_)

# 테스트 데이터에 대한 예측 수행
y_pred = model_random.predict(x_test)




In [ ]:



# 최적의 하이퍼파라미터 확인
print(model_random.best_params_)

# 최고 정확도 확인
print(model_random.best_score_)

# 테스트 데이터에 대한 예측 수행
y_pred = model_random.predict(x_test)



# 테스트 데이터에 대한 예측 수행
y_test_pred = model_random.predict(x_test)

# 테스트 데이터에 예측값을 추가
test_df["quality"] = y_test_pred

# csv 파일로 저장
test_df[["index", "quality"]].to_csv("./data/submission_random_sear_3.csv", index=False)


In [40]:
train_df = pd.read_csv("./data/wine_train_preprocessed.csv")
test_df = pd.read_csv("./data/wine_test_preprocessed.csv")

y_train = train_df["quality"]
x_train = train_df.drop(["index", "quality"], axis=1)
x_test = test_df.drop(["index"], axis=1)

In [16]:
from sklearn.model_selection import StratifiedKFold

straified_kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=1234)

In [17]:
from sklearn.tree import DecisionTreeClassifier

decision_tree_clf = DecisionTreeClassifier(criterion='entropy', random_state=42)
decision_tree_clf.fit(x_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=42)

In [18]:
y_test = decision_tree_clf.predict(x_test)

test_df["quality"] = y_test
test_df[["index", "quality"]].to_csv("./data/submission_limit_max_depth.csv", index=False)

In [19]:
import numpy as np
from numpy.random import randint
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.ensemble import RandomForestClassifier

rs_space={'max_depth':list(np.arange(10, 100, step=10)) + [None],
          'n_estimators':np.arange(10, 500, step=50),
          'max_features':uniform(0,1),
          'criterion':['gini','entropy'],
          'min_samples_leaf':[1, 2, 3, 4],
          'min_samples_split':np.arange(2, 20, step=2)
}

rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(rf, rs_space, n_iter=500, scoring='accuracy', n_jobs=-1, cv=3)
model_random = rf_random.fit(x_train, y_train)






In [20]:
# 최적의 하이퍼파라미터 확인
print(model_random.best_params_)

# 최고 정확도 확인
print(model_random.best_score_)

# 테스트 데이터에 대한 예측 수행
y_pred = model_random.predict(x_test)

# 성능 평가
from sklearn.metrics import accuracy_score
print("Accuracy on test data: ", accuracy_score(y_test, y_pred))


{'criterion': 'gini', 'max_depth': 90, 'max_features': 0.06296189925052409, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 210}
0.6563545495766996
Accuracy on test data:  0.724


In [22]:
# 테스트 데이터에 대한 예측 수행
y_test_pred = model_random.predict(x_test)

# 테스트 데이터에 예측값을 추가
test_df["quality"] = y_test_pred

# csv 파일로 저장
test_df[["index", "quality"]].to_csv("./data/submission_random_search.csv", index=False)


In [ ]:



#와인 품질에 영향을 미치는 요소 파악하기

In [32]:
import numpy as np
from numpy.random import randint
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.ensemble import RandomForestClassifier

rs_space={
    
    
    
  'max_depth': list(np.arange(10, 100, step=10)) + [None],
    'n_estimators': np.arange(100, 1000, step=100),
    'max_features': uniform(loc=0.1, scale=0.9),  # 변경: 범위 축소
    'criterion': ['gini', 'entropy'],
    'min_samples_leaf': np.arange(1, 10),
    'min_samples_split': np.arange(2, 20, step=2)
}

rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(rf, rs_space, n_iter=500, scoring='accuracy', n_jobs=-1, cv=3)
model_random = rf_random.fit(x_train, y_train)






In [33]:
# 최적의 하이퍼파라미터 확인
print(model_random.best_params_)

# 최고 정확도 확인
print(model_random.best_score_)

# 테스트 데이터에 대한 예측 수행
y_pred = model_random.predict(x_test)



{'criterion': 'gini', 'max_depth': 30, 'max_features': 0.30894749763596663, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 600}
0.6536265823639233


In [34]:
# 테스트 데이터에 대한 예측 수행
y_test_pred = model_random.predict(x_test)

# 테스트 데이터에 예측값을 추가
test_df["quality"] = y_test_pred

# csv 파일로 저장
test_df[["index", "quality"]].to_csv("./data/submission_random_sear_3.csv", index=False)


In [46]:
import numpy as np
from numpy.random import randint
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.ensemble import RandomForestClassifier

rs_space={
    
    
    
    'max_depth': list(np.arange(10, 100, step=10)) + [None],
    'n_estimators': np.arange(100, 2000, step=100),
    'max_features': uniform(loc=0.1, scale=0.9),  # 변경: 범위 축소
    'criterion': ['gini', 'entropy'],
    'min_samples_leaf': np.arange(1, 10),
    'min_samples_split': np.arange(2, 20, step=2)
}

rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(rf, rs_space, n_iter=500, scoring='accuracy', n_jobs=-1, cv=3)
model_random = rf_random.fit(x_train, y_train)







KeyboardInterrupt



In [ ]:
# 최적의 하이퍼파라미터 확인
print(model_random.best_params_)

# 최고 정확도 확인
print(model_random.best_score_)

# 테스트 데이터에 대한 예측 수행
y_pred = model_random.predict(x_test)



In [ ]:
# 테스트 데이터에 대한 예측 수행
y_test_pred = model_random.predict(x_test)

# 테스트 데이터에 예측값을 추가
test_df["quality"] = y_test_pred

# csv 파일로 저장
test_df[["index", "quality"]].to_csv("./data/submission_random_sear_4.csv", index=False)


In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# model definition
model = RandomForestClassifier(
    criterion = 'gini',
    n_estimators=210,
    max_depth=90,
    max_features = 3,
    min_samples_leaf=1,
    min_samples_split=4,
    random_state=1234
    
)

# model training
model.fit(x_train, y_train)

# inference
y_pred = model.predict(x_test)

# evaluate model
print("Accuracy: ", accuracy_score(y_test, y_pred))


C:\Users\WOODLAC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: 'white'

In [50]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# model definition
model = RandomForestClassifier(
    criterion = 'gini',
    n_estimators=210,
    max_depth=90,
    max_features = 3,
    min_samples_leaf=1,
    min_samples_split=4,
    random_state=1234
    
)

# model training
model.fit(x_train, y_train)

# inference
y_pred = model.predict(x_test)

# evaluate model
print("Accuracy: ", accuracy_score(y_test, y_pred))


C:\Users\WOODLAC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: 'white'

In [64]:
import pandas as pd

# Create DataFrame from prediction results
df = pd.DataFrame({
    "index": x_test.index,  # assuming x_test is a DataFrame and has an index
    "quality": y_pred
})

# Save DataFrame to csv
df.to_csv('./data/predictions_3.csv', index=False)


In [70]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(
    criterion = 'gini',
    n_estimators = 400,
    max_depth = 80,
    max_features = 5,
    min_samples_leaf = 1,
    min_samples_split = 2,
    random_state = 1234
)

# model training
model.fit(x_train, y_train)

# inference
y_pred = model.predict(x_test)

# evaluate model
print("Accuracy: ", accuracy_score(y_test, y_pred))


Accuracy:  0.737


In [71]:
import pandas as pd

# Create DataFrame from prediction results
df = pd.DataFrame({
    "index": x_test.index,  # assuming x_test is a DataFrame and has an index
    "quality": y_pred
})

# Save DataFrame to csv
df.to_csv('./data/predictions_2.csv', index=False)


In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

label_encoder = LabelEncoder()

def label_encode_type(df):
    df["type"] = label_encoder.fit_transform(df["type"]) 
    return df

def preprocess(df):
    df = label_encode_type(df)
    return df

preprocessed_train_df = preprocess(train_df)
preprocessed_test_df = preprocess(test_df)

preprocessed_train_df.to_csv('./data/wine_train_preprocessed.csv', index=False)
preprocessed_test_df.to_csv('./data/wine_test_preprocessed.csv', index=False)

train_df = pd.read_csv('./data/wine_train_preprocessed.csv')
test_df = pd.read_csv('./data/wine_test_preprocessed.csv')

y_train = train_df["quality"]
x_train = train_df.drop(["index", "quality"], axis=1)
x_test = test_df.drop(["index"], axis=1)

model = RandomForestClassifier(
    criterion='gini',
    n_estimators=210,
    max_depth=90,
    max_features=3,
    min_samples_leaf=1,
    min_samples_split=4,
    random_state=1234
)

scores = cross_val_score(model, x_train,y_train,cv=straified_kf)
scores, scores.mean()

model.fit(x_train,y_train)

y_test = model.predict(x_test)
test_df["quality"] = y_test
test_df[["index", "quality"]].to_csv('./data/testttt.csv', index=False)